In [ ]:
import keras
import os
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow_hub as hub
from tensorflow.keras import layers
from sklearn.model_selection import KFold

In [ ]:
%pip install ogb torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 27.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=6159a820133c9a243e3a21ef7bc992c6a8b3dfe8e3d8b269bb7e2049432c4eea
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=bdcbf700a1ab65d57ffe877b9b06e232ad3c0c698ddc716804d6b774110074c6
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built tor

In [ ]:
import ogb
from ogb.nodeproppred import NodePropPredDataset 
from torch_geometric.data import DataLoader

#OGBN Mag

In [ ]:

#H:\\PhD Research\Data\PROTEINS\PROTEINS_node_attributes.txt
# Download and process data at './dataset/ogbg_molhiv/'
dataset = NodePropPredDataset(name = "ogbn-mag", root = '/Users/joshem/PhD Research/Data/')
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"]['paper'], split_idx["valid"]['paper'], split_idx["test"]['paper']
graph = dataset[0] # pyg graph object

Node_Class = graph[1]['paper'][:,0]
Node_Year=graph[0]['node_year']['paper']
E=graph[0]['edge_index_dict'][('paper', 'cites', 'paper')]
n = graph[0]['num_nodes_dict']['paper']

F_vec_url = 'https://utdallas.box.com/shared/static/19yvbjlxzkwhjft5k9r9sfu23pkvm4aa.zip'
F_vec_file = '/content/Feature_nbd_mag.csv'
Node_Fec = list(graph[0]['node_feat_dict']['paper'])
Node_Year_list=list(Node_Year[:,0])
num_cls = max(Node_Class) + 1

Downloaded 0.40 GB: 100%|██████████| 413/413 [00:30<00:00, 13.52it/s]


Extracting /Users/joshem/PhD Research/Data/mag.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 7810.62it/s]

Saving...


#OGBN Arxiv

In [ ]:
dataset = NodePropPredDataset(name = "ogbn-arxiv", root = '/Users/joshem/PhD Research/Data/')
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph = dataset[0] # pyg graph object

Node_Class = graph[1][:,0]
Node_Year=graph[0]['node_year']
E=graph[0]['edge_index']
n = graph[0]['num_nodes']

F_vec_url = 'https://utdallas.box.com/shared/static/4tsidulh34mrqa1fiemejvk7xckp9not.zip'
F_vec_file = '/content/Feature_OGB_Arxiv.csv'
Node_Fec = list(graph[0]['node_feat'])
Node_Year_list=list(Node_Year[:,0])
num_cls = max(Node_Class)+1

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:08<00:00,  9.03it/s]


Extracting /Users/joshem/PhD Research/Data/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 10565.00it/s]

Saving...


# OGBN Proteins

In [ ]:
dataset = NodePropPredDataset(name = "ogbn-proteins", root = 'H:\\PhD Research\Python code\OGB')
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph = dataset[0] # pyg graph object

Node_Class = graph[1]
E = graph[0]['edge_index']
n = graph[0]['num_nodes']
num_cls = Node_Class.shape[1]

Downloaded 0.21 GB: 100%|██████████| 216/216 [00:18<00:00, 11.67it/s]


Extracting H:\PhD Research\Python code\OGB/proteins.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


Saving...


In [ ]:
n

736389

In [ ]:
data = pd.read_csv('https://utdallas.box.com/shared/static/ukzv4nd8jjg4y8zqb4y66ekmhw8fmfvj.csv', index_col=0)

In [ ]:
data.head()

,1,2,3,4,5,6,7,8,9,10,...,999,1000,1001,1002,1003,1004,1005,1006,1007,1008
0,311.0,308.0,235.0,281.0,275.0,128.0,272.0,113.0,256.0,187.0,...,5.979,0.008,0.003,0.003,0.462,0.983,0.003,2.432,0.000,0.0
1,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,...,0.000,0.000,0.000,0.000,0.968,0.000,0.000,0.000,0.000,0.0
2,105.0,89.0,70.0,62.0,66.0,34.0,49.0,31.0,44.0,39.0,...,1.133,1.122,0.000,0.581,5.428,0.465,0.000,0.584,0.000,0.0
3,426.0,412.0,307.0,372.0,350.0,210.0,333.0,191.0,322.0,240.0,...,8.860,10.153,0.000,1.170,1.523,3.057,0.000,2.335,0.000,0.0
4,360.0,290.0,258.0,229.0,241.0,139.0,166.0,133.0,161.0,178.0,...,5.963,3.508,0.000,1.496,0.680,4.017,0.000,0.714,0.152,0.0


In [ ]:
graph

({'edge_index_dict': {('author',
    'affiliated_with',
    'institution'): array([[      0,       1,       2, ..., 1134645, 1134647, 1134648],
          [    845,     996,    3197, ...,    5189,    4668,    4668]]),
   ('author',
    'writes',
    'paper'): array([[      0,       0,       0, ..., 1134647, 1134648, 1134648],
          [  19703,  289285,  311768, ...,  657395,  671118,  719594]]),
   ('paper',
    'cites',
    'paper'): array([[     0,      0,      0, ..., 736388, 736388, 736388],
          [    88,  27449, 121051, ..., 421711, 427339, 439864]]),
   ('paper',
    'has_topic',
    'field_of_study'): array([[     0,      0,      0, ..., 736388, 736388, 736388],
          [   145,   2215,   3205, ...,  21458,  22283,  31934]])},
  'edge_feat_dict': None,
  'node_feat_dict': {'paper': array([[-0.095379,  0.040758, -0.210948, ...,  0.061569, -0.027663,
           -0.133832],
          [-0.151047, -0.107315, -0.221964, ...,  0.345754, -0.027737,
           -0.218527],
       

# Creating Neighborhood Feature Vectors

In [ ]:
EdgeL=E[0]
EdgeR=E[1]
print(EdgeL)
print(EdgeR)

[     0      0      0 ... 736388 736388 736388]
[    88  27449 121051 ... 421711 427339 439864]


In [ ]:
N_class_train= Node_Class.copy()
for d in test_idx:
    N_class_train[d]=500
print(N_class_train)

[246 131 189 ... 266 289   1]


In [ ]:
EdgeL[0]

0

In [ ]:
np.where(EdgeR==0)

(array([    397, 1670849]),)

In [ ]:
max(Node_Class)

348

In [ ]:
#@title For Mag and Arxiv
import time
start_time = time.time()
F_vec = np.zeros((n, num_cls*2))
for i in range(len(EdgeL)):
  print("\rProcessing file {} ({}%)".format(i, 100*i//len(EdgeL)), end='', flush=True)
  left, right = EdgeL[i], EdgeR[i]
  F_vec[left][Node_Class[right]*2] += 1
  F_vec[right][Node_Class[left]*2+1] += 1
end_time = time.time()
print(f'--- {(end_time-start_time)/3600} hours ---')

Processing file 14785 (0%)

KeyboardInterrupt: ignored

In [ ]:
graph[0]['edge_index_dict'][('author', 'writes', 'paper')].shape[1]

7145660

In [ ]:
author_edges = graph[0]['edge_index_dict'][('author', 'writes', 'paper')]
A_Edge, P_Edge = author_edges
num_authors = graph[0]['num_nodes_dict']['author']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
len(graph[0]['edge_feat'])

79122504

In [ ]:
graph[0]['edge_feat'][0]

array([0.501, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001],
      dtype=float32)

In [ ]:
#@title For Proteins
import time
start_time = time.time()
E_feats = graph[0]['edge_feat']
F_vec = [[0 for _ in range(num_cls*9)] for _ in range(n)]
for i in range(0, len(EdgeL), 2):
  print("\rProcessing file {} ({}%)".format(i, 100*i//len(EdgeL)), end='', flush=True)
  e_feat = np.insert(E_feats[i], 0, 1)
  left, right = EdgeL[i], EdgeR[i]
  vec1 = np.concatenate([Node_Class[right] * e_f for e_f in e_feat])
  vec2 = np.concatenate([Node_Class[left] * e_f for e_f in e_feat])
  F_vec[left] += vec1
  F_vec[right] += vec2

end_time = time.time()
print(f'--- {(end_time-start_time)/3600} hours ---')

Processing file 412966 (0%)

KeyboardInterrupt: ignored

In [ ]:
def CountX(lst,x):
    return(lst.count(x))

start_time = time.time()
F_vec=[]
for i in range(n):
    print("\rProcessing file {} ({}%)".format(i, 100*i//n), end='', flush=True)
    node_F=[]
    list_out=[]
    list_In=[]
    #S_nbd_out=[]
    #S_nbd_in=[]
    indx=np.where(EdgeL==i)
    indxIn=np.where(EdgeR==i)
    #print(indx)
    for j in indx[0]:
        list_out.append(N_class_train[EdgeR[j]])
   # indx[0].clear()
    for k in indxIn[0]:
        list_In.append(N_class_train[EdgeL[k]])
   # indxIn[0].clear()

    # print(indx)
    # print(indxIn)
    # print(list_out)
    # print(list_In)
                
                
    for d in range(max(Node_Class)+1):
        node_F.append(CountX(list_out,d))
        node_F.append(CountX(list_In,d))
    F_vec.append(node_F)
print(np.where(np.array(F_vec[0])!=0))
data = pd.DataFrame(F_vec)
# data.insert(loc=int(num_cls*2), column='Class', value=Node_Class)
end_time = time.time()
print(f'--- {(end_time-start_time)/3600} hours ---')

Processing file 0 (0%)(array([0, 1, 2, 3, 4]),)
(array([    397, 1670849]),)
[246, 309, 146, 309, 324]
[246, 146]
(array([292, 293, 492, 493, 618, 648]),)
--- 1.8209682570563423e-05 hours ---


# Retrieving Saved Neighborhood Feature Vectors

In [ ]:
import requests, zipfile, io
r = requests.get(F_vec_url, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('/content')

In [ ]:
data = pd.read_csv(F_vec_file, index_col=0)

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,689,690,691,692,693,694,695,696,697,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,246
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,131
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,189
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,131
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,95


In [ ]:
import time
start_time = time.time()

author_vecs = np.zeros((num_authors, num_cls))
for i in range(len(A_Edge)):
  print("\rProcessing file {} ({}%)".format(i, 100*i//len(A_Edge)), end='', flush=True)
  auth, paper = A_Edge[i], P_Edge[i]
  if N_class_train[paper] == 500:
    continue
  author_vecs[auth][Node_Class[paper]] += 1
# for i in range(len(author_vecs)):
#   author_vecs[i] /= np.linalg.norm(author_vecs, ord=1)
compression_opts = dict(method='zip',
                        archive_name='Author_vecs.csv')  
author_df = pd.DataFrame(author_vecs)
author_df.to_csv('/content/drive/MyDrive/Author_vecs.zip', compression=compression_opts)

end_time = time.time()
print(f'--- {(end_time-start_time)/3600} hours ---')

Processing file 4037518 (56%)

In [ ]:
#@title Reading Author vecs
z = zipfile.ZipFile('/content/drive/MyDrive/Author_vecs.zip')
z.extractall('/content')
author_df = pd.read_csv('/content/Author_vecs.csv', index_col=0)

In [ ]:
author_vecs = author_df.to_numpy()

In [ ]:
author_vecs.shape

(1134649, 349)

In [ ]:
z = zipfile.ZipFile('/content/drive/MyDrive/Topic_vecs.zip')
z.extractall('/content')
topic_df = pd.read_csv('/content/Topic_vecs.csv', index_col=0)

In [ ]:
topic_vecs = topic_df.to_numpy()

In [ ]:
data['Class'] = Node_Class

In [ ]:
z = zipfile.ZipFile('/content/drive/MyDrive/Paper_topic_vecs_max.zip')
z.extractall('/content')
paper_topic_df = pd.read_csv('/content/Paper_topic_vecs_max.csv', index_col=0)

In [ ]:
z = zipfile.ZipFile('/content/drive/MyDrive/Paper_auth_vecs_max.zip')
z.extractall('/content')
paper_auth_df = pd.read_csv('/content/Paper_auth_vecs_max.csv', index_col=0)

In [ ]:
max(paper_topic_df['0'])

2131.0

#Creating Node Description Feature Vectors

In [ ]:
feature_names = ["w_{}".format(ii) for ii in range(len(Node_Fec[0]))]
Node_Fec = pd.DataFrame(Node_Fec,columns=feature_names)

In [ ]:
Node_Fec.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_118,w_119,w_120,w_121,w_122,w_123,w_124,w_125,w_126,w_127
0,-0.057943,-0.052530,-0.072603,-0.026555,0.130435,-0.241386,-0.449242,-0.018443,-0.087218,0.112320,...,0.211490,-0.226118,-0.185603,0.053230,0.332873,0.104175,0.007408,0.173364,-0.172796,-0.140059
1,-0.124500,-0.070665,-0.325202,0.007779,-0.001559,0.074189,-0.191013,0.049689,0.026369,0.099364,...,0.106316,0.052926,-0.258378,0.021567,0.281503,-0.173423,0.202082,0.068524,-0.372111,-0.301036
2,-0.080242,-0.023328,-0.183787,-0.180707,0.075765,-0.125818,-0.394573,-0.219078,-0.108931,0.056966,...,0.019453,-0.070291,-0.177562,-0.214012,0.182186,-0.121589,-0.073642,0.109919,0.117589,-0.139883
3,-0.145044,0.054915,-0.126666,0.039971,-0.055909,-0.101278,-0.339202,-0.115801,-0.080058,-0.001633,...,-0.065752,0.042735,0.066338,-0.226921,0.188418,-0.017295,0.063449,0.017816,0.085364,-0.081804
4,-0.071154,0.070766,-0.281432,-0.161892,-0.165246,-0.029116,-0.338593,-0.138727,0.100015,0.132794,...,-0.056130,0.047475,-0.263795,0.026462,0.376349,-0.253772,0.084472,0.098033,-0.075347,-0.111687


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaling = StandardScaler()

Scaled_data = Node_Fec

scaling.fit(Scaled_data)
Scaled_data = scaling.transform(Scaled_data)

m = 50
principal = PCA(n_components=m)
principal.fit(Scaled_data)
x = principal.transform(Scaled_data)

print(x.shape)

(169343, 50)


In [ ]:
Scaled_data = pd.DataFrame(x)

In [ ]:
# Replace Node_Fec with Scaled_data if you want to use PCA
Data = pd.concat([data, Node_Fec], axis=1)

In [ ]:
Data = pd.concat([data, Scaled_data], axis=1)

In [ ]:
edge_feat_df = pd.read_csv('https://utdallas.box.com/shared/static/hju4cvgquks1a2wijtbp9obiijldl0c3.csv', index_col=0)

In [ ]:
edge_feat_df = edge_feat_df.iloc[:, :16]

In [ ]:
edge_feat_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,471,2,4,6,10,6,11,1,20.511,1.292,0.511,0.511,47.750,39.328,0.511000,47.564
1,7,0,0,0,0,0,0,0,0.007,0.007,0.007,0.007,0.007,0.007,0.007000,5.286
2,184,0,0,0,0,0,0,0,0.184,0.184,0.772,0.184,20.474,5.423,0.184000,29.872
3,731,1,1,1,1,1,1,0,3.737,0.737,1.541,0.737,75.981,95.882,160.577004,91.906
4,591,0,0,0,0,0,0,0,0.591,0.591,0.591,0.591,100.085,1.376,0.591000,43.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132529,0,0,6,1,1,3,4,0,7.015,0.015,0.015,0.224,0.015,0.015,0.015000,0.015
132530,0,0,6,1,1,3,4,0,7.015,0.015,0.015,0.228,0.015,0.015,0.015000,0.015
132531,0,0,6,1,1,3,4,0,7.015,0.015,0.015,0.227,0.015,0.015,0.015000,0.015
132532,0,0,6,1,1,3,4,0,7.015,0.015,0.015,0.222,0.015,0.015,0.015000,0.015


In [ ]:
Node_Class_df = pd.DataFrame(Node_Class)

In [ ]:
Data = pd.concat([data, Node_Class_df], axis=1)

In [ ]:
Data.head()

,1,2,3,4,5,6,7,8,9,10,...,102,103,104,105,106,107,108,109,110,111
0,311.0,308.0,235.0,281.0,275.0,128.0,272.0,113.0,256.0,187.0,...,0,0,0,0,0,0,0,0,0,0
1,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,...,0,0,0,0,0,0,0,0,0,0
2,105.0,89.0,70.0,62.0,66.0,34.0,49.0,31.0,44.0,39.0,...,0,0,0,0,0,0,0,0,0,0
3,426.0,412.0,307.0,372.0,350.0,210.0,333.0,191.0,322.0,240.0,...,0,0,0,0,0,0,0,0,0,0
4,360.0,290.0,258.0,229.0,241.0,139.0,166.0,133.0,161.0,178.0,...,0,1,0,0,1,0,0,0,0,0


In [ ]:
len(P_Edge)

7145660

In [ ]:
P_T_Edges = graph[0]['edge_index_dict'][('paper', 'has_topic', 'field_of_study')]

In [ ]:
num_fields = graph[0]['num_nodes_dict']['field_of_study']

In [ ]:
P_Edges, T_Edges = P_T_Edges

In [ ]:
import time
start_time = time.time()

topic_vecs = np.zeros((num_fields, num_cls))
for i in range(len(P_Edges)):
  print("\rProcessing file {} ({}%)".format(i, 100*i//len(P_Edges)), end='', flush=True)
  topic, paper = T_Edges[i], P_Edges[i]
  if N_class_train[paper] == 500:
    continue
  topic_vecs[topic][Node_Class[paper]] += 1
# for i in range(len(author_vecs)):
#   author_vecs[i] /= np.linalg.norm(author_vecs, ord=1)
compression_opts = dict(method='zip',
                        archive_name='Topic_vecs.csv')  
topic_df = pd.DataFrame(topic_vecs)
topic_df.to_csv('/content/drive/MyDrive/Topic_vecs.zip', compression=compression_opts)

end_time = time.time()
print(f'--- {(end_time-start_time)/3600} hours ---')

Processing file 7505077 (99%)--- 2.806017263929049 hours ---


In [ ]:
from collections import Counter
import time

start_time = time.time()

paper_topic_vecs = np.zeros((n, num_cls))

# topic_sums = [sum(topic_vecs[topic]) for topic in range(len(topic_vecs))]
# paper_topic_sums = [0 for _ in range(n)]
for i in range(len(P_Edges)):
  print("\rProcessing file {} ({}%)".format(i, 100*i//len(P_Edges)), end='', flush=True)
  paper, topic = P_Edges[i], T_Edges[i]
  paper_topic_vecs[paper] += topic_vecs[topic]
print()
# for i in range(n):
#   print("\rProcessing file {} ({}%)".format(i, 100*i//n), end='', flush=True)
#   paper_auth_vecs[i] /= paper_counts[i]

paper_topic_df = pd.DataFrame(paper_topic_vecs)
compression_opts = dict(method='zip',
                        archive_name='Paper_topic_vecs_max.csv')
paper_topic_df.to_csv('/content/drive/MyDrive/Paper_topic_vecs_max.zip', compression=compression_opts)

end_time = time.time()
print(f'--- {(end_time-start_time)/3600} hours ---')

Processing file 892491 (11%)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing file 7505027 (99%)--- 2.785127653744486 hours ---


In [ ]:
from collections import Counter
import time

start_time = time.time()

paper_auth_vecs = np.zeros((n, num_cls))
paper_counts = Counter(P_Edge)

auth_sums = [sum(author_vecs[author]) for author in range(len(author_vecs))]
paper_auth_sums = [0 for _ in range(n)]
for i in range(len(P_Edge)):
  print("\rProcessing file {} ({}%)".format(i, 100*i//len(P_Edge)), end='', flush=True)
  paper, author = P_Edge[i], A_Edge[i]
  if auth_sums[author] > paper_auth_sums[paper]:
    paper_auth_vecs[paper] = author_vecs[author]
    paper_auth_sums[paper] = auth_sums[author]
print()
# for i in range(n):
#   print("\rProcessing file {} ({}%)".format(i, 100*i//n), end='', flush=True)
#   paper_auth_vecs[i] /= paper_counts[i]

paper_auth_df = pd.DataFrame(paper_auth_vecs)
compression_opts = dict(method='zip',
                        archive_name='Paper_auth_vecs_max.csv')
paper_auth_df.to_csv('/content/drive/MyDrive/Paper_auth_vecs_max.zip', compression=compression_opts)

end_time = time.time()
print(f'--- {(end_time-start_time)/3600} hours ---')

Processing file 7145656 (99%)--- 2.711470737722185 hours ---


In [ ]:
import zipfile
z = zipfile.ZipFile('/content/Mag_F_vecs.zip')
z.extractall('/content')

In [ ]:
Data = pd.read_csv('/content/Mag_F_vecs.csv')

In [ ]:
paper_auth_df

,0,1,2,3,4,5,6,7,8,9,...,339,340,341,342,343,344,345,346,347,348
0,0.000000,0.682927,0.0,0.017422,0.000000,0.000000,0.017422,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.003484,0.0,0.0,0.006969,0.006969,0.0
1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736384,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
736385,0.000734,0.105727,0.0,0.000000,0.000734,0.002937,0.011747,0.000734,0.0,0.008076,...,0.0,0.0,0.0,0.002203,0.000734,0.0,0.0,0.015419,0.031571,0.0
736386,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
736387,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,1.250000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [ ]:
graph[0]

{'edge_index_dict': {('author',
   'affiliated_with',
   'institution'): array([[      0,       1,       2, ..., 1134645, 1134647, 1134648],
         [    845,     996,    3197, ...,    5189,    4668,    4668]]),
  ('author',
   'writes',
   'paper'): array([[      0,       0,       0, ..., 1134647, 1134648, 1134648],
         [  19703,  289285,  311768, ...,  657395,  671118,  719594]]),
  ('paper',
   'cites',
   'paper'): array([[     0,      0,      0, ..., 736388, 736388, 736388],
         [    88,  27449, 121051, ..., 421711, 427339, 439864]]),
  ('paper',
   'has_topic',
   'field_of_study'): array([[     0,      0,      0, ..., 736388, 736388, 736388],
         [   145,   2215,   3205, ...,  21458,  22283,  31934]])},
 'edge_feat_dict': None,
 'node_feat_dict': {'paper': array([[-0.095379,  0.040758, -0.210948, ...,  0.061569, -0.027663,
          -0.133832],
         [-0.151047, -0.107315, -0.221964, ...,  0.345754, -0.027737,
          -0.218527],
         [-0.114799, -0.1759

In [ ]:
Data = pd.concat([data, Node_Fec, paper_auth_df, paper_topic_df], axis=1)

In [ ]:
paper_auth_df

,0,1,2,3,4,5,6,7,8,9,...,339,340,341,342,343,344,345,346,347,348
0,0.0,51.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
736385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
736386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
736387,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Data

,0,1,2,3,4,5,6,7,8,9,...,339,340,341,342,343,344,345,346,347,348
0,0,0,0,0,0,0,0,0,0,0,...,219.0,315.0,137.0,263.0,308.0,284.0,209.0,240.0,522.0,301.0
1,0,0,0,0,0,0,0,0,0,0,...,205.0,317.0,139.0,268.0,194.0,261.0,211.0,213.0,438.0,301.0
2,0,0,0,0,0,0,0,0,0,0,...,240.0,311.0,137.0,261.0,194.0,249.0,214.0,299.0,836.0,306.0
3,0,0,0,0,0,0,0,0,0,0,...,208.0,320.0,139.0,268.0,197.0,302.0,212.0,213.0,455.0,304.0
4,0,0,0,0,0,0,0,0,0,0,...,244.0,322.0,146.0,288.0,195.0,251.0,215.0,297.0,1031.0,312.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736384,0,0,0,0,0,0,0,0,0,0,...,207.0,332.0,139.0,280.0,191.0,240.0,248.0,213.0,439.0,327.0
736385,0,0,1,0,0,0,0,0,0,0,...,238.0,309.0,137.0,257.0,195.0,237.0,210.0,302.0,910.0,297.0
736386,0,0,0,0,0,0,0,0,0,0,...,211.0,594.0,144.0,359.0,194.0,290.0,291.0,219.0,442.0,371.0
736387,0,0,0,0,0,0,0,0,0,0,...,204.0,317.0,177.0,335.0,191.0,240.0,221.0,214.0,430.0,307.0


In [ ]:
del data, Node_Fec, paper_auth_df, paper_topic_df

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='Mag_F_vecs.csv')
Data.to_csv('/content/drive/MyDrive/Mag_F_vecs.zip', compression=compression_opts)

In [ ]:
Data = Data.drop(Data.columns[0], axis=1)

In [ ]:
paper_counts

Counter({19703: 5,
         289285: 4,
         311768: 5,
         402711: 3,
         181505: 3,
         297095: 3,
         336569: 3,
         14217: 6,
         14630: 6,
         15376: 3,
         15420: 5,
         20656: 9,
         27715: 3,
         37354: 3,
         62987: 3,
         78157: 4,
         86646: 13,
         91551: 9,
         103412: 8,
         129861: 5,
         148900: 7,
         178805: 4,
         190892: 5,
         194646: 5,
         261498: 2,
         301784: 3,
         357433: 9,
         432992: 5,
         455787: 7,
         489838: 5,
         519995: 16,
         540904: 8,
         599199: 3,
         600495: 9,
         614248: 6,
         617244: 3,
         647287: 4,
         689138: 5,
         705788: 7,
         722195: 6,
         729975: 3,
         230166: 2,
         392216: 2,
         504678: 2,
         625373: 6,
         240904: 2,
         393817: 3,
         499283: 4,
         538604: 4,
         621886: 5,
         7

# Training

In [ ]:
X = Data.drop('Class', axis=1)
y = Data['Class']
X_train=X.iloc[list(train_idx)+list(valid_idx)]
X_test=X.iloc[test_idx]
y_train=y.iloc[list(train_idx)+list(valid_idx)]
y_test=y.iloc[test_idx]

In [ ]:
del X, y

In [ ]:
del topic_df, topic_vecs, paper_topic_vecs

In [ ]:
X = Data.iloc[:, :-112]
y = Data.iloc[:, -112:]
X_train=X.iloc[list(train_idx)+list(valid_idx)]
X_test=X.iloc[test_idx].drop(132533)
y_train=y.iloc[list(train_idx)+list(valid_idx)]
y_test=y.iloc[test_idx].drop(132533)

In [ ]:
y_2 = Data.iloc[:, -112:-109]
y_train_2=y_2.iloc[list(train_idx)+list(valid_idx)]
y_test_2=y_2.iloc[test_idx].drop(132533)

In [ ]:
y_2

,0,1,2
0,1,1,1
1,1,1,0
2,0,0,0
3,0,0,0
4,1,1,1
...,...,...,...
132529,0,0,0
132530,0,0,0
132531,0,0,0
132532,0,0,0


In [ ]:
y_test

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
107855,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107857,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107858,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132529,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132531,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
earlystop1 = EarlyStopping(patience=7)
earlystop2 = ReduceLROnPlateau(monitor='val_loss',
                               min_lr = 3e-7, 
                               patience = 4,
                               factor=0.3,
                               verbose = 1)

In [ ]:
%pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.6/528.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 48.0 MB/s eta 0:00:00


In [ ]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
checkpoint_filepath='/tmp/checkpoint'
# for train, test in kfold.split(X_train, y_train):
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(X_train.iloc[0]),)),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(len(y.columns), activation='sigmoid')
])

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=tf.keras.optimizers.AdamW(2e-5), loss=loss_fn, 
              metrics=[tf.keras.metrics.AUC(name='auc'), 'binary_accuracy'])
print('------------------------------------------------------------------------')
print(f'Training for fold {fold_no} ...')
history = model.fit(np.array(X_train), np.array(y_train),
                    batch_size = 128,
                    epochs=100,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=(np.array(X_test), np.array(y_test)),
                    callbacks=[
                                tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                save_best_only=True,
                                                                save_weights_only=True,
                                                                monitor='val_auc',
                                                                mode='max')])
model.load_weights(checkpoint_filepath)
scores = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])
fold_no += 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
843/843 [==============================] - 5s 4ms/step - loss: 8.4430 - auc: 0.5208 - binary_accuracy: 0.7017 - val_loss: 0.6481 - val_auc: 0.5342 - val_binary_accuracy: 0.8669
Epoch 2/100
843/843 [==============================] - 3s 4ms/step - loss: 3.4946 - auc: 0.5324 - binary_accuracy: 0.7459 - val_loss: 0.4648 - val_auc: 0.5415 - val_binary_accuracy: 0.8976
Epoch 3/100
843/843 [==============================] - 3s 4ms/step - loss: 2.5278 - auc: 0.5381 - binary_accuracy: 0.7600 - val_loss: 0.4044 - val_auc: 0.5432 - val_binary_accuracy: 0.9096
Epoch 4/100
843/843 [==============================] - 3s 4ms/step - loss: 2.0151 - auc: 0.5423 - binary_accuracy: 0.7692 - val_loss: 0.3585 - val_auc: 0.5453 - val_binary_accuracy: 0.9172
Epoch 5/100
843/843 [==============================] - 3s 4ms/step - loss: 1.6964 - auc: 0.5464 - binary_accuracy: 0.7776 - val_loss: 0.3446 - val_

In [ ]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
checkpoint_filepath='/tmp/checkpoint'
# for train, test in kfold.split(X_train, y_train):
model_2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(X_train.iloc[0]),)),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(len(y_2.columns), activation='sigmoid')
])

loss_fn = tf.keras.losses.BinaryCrossentropy()
model_2.compile(optimizer=tf.keras.optimizers.Adam(2e-5), loss=loss_fn, 
              metrics=[tf.keras.metrics.AUC(name='auc')])
print('------------------------------------------------------------------------')
print(f'Training for fold {fold_no} ...')
history = model_2.fit(np.array(X_train), np.array(y_train_2),
                    batch_size = 128,
                    epochs=100,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=(np.array(X_test), np.array(y_test_2)),
                    callbacks=[
                                tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                save_best_only=True,
                                                                save_weights_only=True,
                                                                monitor='val_auc',
                                                                mode='max')])
model_2.load_weights(checkpoint_filepath)
scores = model_2.evaluate(np.array(X_test), np.array(y_test_2), verbose=0)
print(f'Score for fold {fold_no}: {model_2.metrics_names[0]} of {scores[0]}; {model_2.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])
fold_no += 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
843/843 [==============================] - 5s 4ms/step - loss: 15.7280 - auc: 0.5758 - val_loss: 4.7970 - val_auc: 0.5306
Epoch 2/100
843/843 [==============================] - 3s 3ms/step - loss: 8.5744 - auc: 0.5887 - val_loss: 4.1395 - val_auc: 0.5340
Epoch 3/100
843/843 [==============================] - 3s 3ms/step - loss: 6.0498 - auc: 0.5938 - val_loss: 3.9661 - val_auc: 0.5330
Epoch 4/100
843/843 [==============================] - 3s 4ms/step - loss: 4.8432 - auc: 0.5977 - val_loss: 2.1316 - val_auc: 0.5388
Epoch 5/100
843/843 [==============================] - 3s 3ms/step - loss: 4.0415 - auc: 0.6020 - val_loss: 2.3948 - val_auc: 0.5160
Epoch 6/100
843/843 [==============================] - 3s 3ms/step - loss: 3.4951 - auc: 0.6033 - val_loss: 1.6346 - val_auc: 0.5311
Epoch 7/100
843/843 [==============================] - 3s 3ms/step - loss: 3.0409 - auc: 0.6062 - val_lo

KeyboardInterrupt: ignored

In [ ]:
y_pred = model.predict(X_test)

772/772 [==============================] - 1s 1ms/step


In [ ]:
y_pred = np.round(y_pred)

In [ ]:
y_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
y_test

,T_1,T_2,T_3,T_4,T_5,T_6,T_7,T_8,T_9,T_10,...,T_103,T_104,T_105,T_106,T_107,T_108,T_109,T_110,T_111,T_112
107855,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107857,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107858,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132529,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132531,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics import roc_auc_score

for i in range(112):
  print(roc_auc_score(y_test.to_numpy()[:, i], y_pred[:, i]))

0.49877402406155447
0.5
0.499982376313368
0.5
0.5
0.5
0.5
0.5
0.5016193553774021
0.5
0.5003491638879127
0.5
0.5
0.5
0.5
0.4998967240880737
0.5
0.49981198295312107
0.5
0.4998538072762207
0.5
0.49997906898861355
0.5
0.5
0.49977044117033265
0.5
0.5
0.5
0.49995833159714986
0.5
0.5
0.5
0.5
0.49991701933449506
0.5
0.4999377722464219
0.5
0.5
0.5
0.5013951002092557
0.4998754463173628
0.5
0.5
0.5
0.5
0.5
0.49991695731606045
0.4999585199933632
0.5066979644536855
0.5
0.5
0.5151883343668864
0.5
0.49997947202036375
0.4999582341394144
0.4998967283542631
0.5
0.49991808985542857
0.5
0.5
0.5
0.5
0.4999379755210056
0.5
0.5
0.4999793191876577
0.4998969454635393
0.4998969454635393
0.5
0.49962796081187216
0.5
0.5
0.5
0.4999180596525729
0.5
0.5
0.5005623860297338
0.5
0.5
0.5
0.5
0.4999386025950637
0.5
0.49944341372912804
0.5060164330946612
0.5
0.49909304530650944
0.5
0.5
0.4999591252810137
0.5
0.5
0.5
0.5
0.5
0.5
0.49971381847914964
0.5
0.49989785078042004
0.5
0.5
0.5
0.49530151985618565
0.49995914031216804

In [ ]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
checkpoint_filepath='/tmp/checkpoint'
# for train, test in kfold.split(X_train, y_train):
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(X_train.iloc[0]),)),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(max(y_train)+1, activation='softmax')
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(7e-5), loss=loss_fn, 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
print('------------------------------------------------------------------------')
print(f'Training for fold {fold_no} ...')
history = model.fit(np.array(X_train), np.array(y_train),
                    batch_size = 128,
                    epochs=100,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=(np.array(X_test), np.array(y_test)),
                    callbacks=[earlystop1, earlystop2,
                                tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                save_best_only=True,
                                                                save_weights_only=True,
                                                                monitor='val_sparse_categorical_accuracy',
                                                                mode='max')])
model.load_weights(checkpoint_filepath)
scores = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])
fold_no += 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
944/944 [==============================] - 5s 4ms/step - loss: 1.5910 - sparse_categorical_accuracy: 0.6545 - val_loss: 1.2088 - val_sparse_categorical_accuracy: 0.7018 - lr: 7.0000e-05
Epoch 2/100
944/944 [==============================] - 3s 4ms/step - loss: 1.2293 - sparse_categorical_accuracy: 0.7104 - val_loss: 1.1475 - val_sparse_categorical_accuracy: 0.7038 - lr: 7.0000e-05
Epoch 3/100
944/944 [==============================] - 3s 4ms/step - loss: 1.1740 - sparse_categorical_accuracy: 0.7155 - val_loss: 1.1166 - val_sparse_categorical_accuracy: 0.7030 - lr: 7.0000e-05
Epoch 4/100
944/944 [==============================] - 3s 4ms/step - loss: 1.1236 - sparse_categorical_accuracy: 0.7168 - val_loss: 1.0768 - val_sparse_categorical_accuracy: 0.7144 - lr: 7.0000e-05
Epoch 5/100
944/944 [==============================] - 3s 4ms/step - loss: 1.1112 - sparse_categorical_accuracy